In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, string
import csv
import pymongo
import time
import json 
import numpy as np

# Extract (Scrape) Data

#### *Does not need to be loaded to pymongo* - can be scraped on the spot for the Project 2

In [ ]:
# currentDirectory = os.path.dirname(__file__)
# executable_path = {"executable_path": f"{currentDirectory}\chromedriver.exe"}
executable_path = {'executable_path': 'chromedriver.exe'} #notebook version
browser = Browser('chrome', **executable_path, headless=True)

### Scrape Schedules

In [ ]:
nba_sched_url = "https://www.espn.com/nba/schedule/_/date/"
browser.visit(nba_sched_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')
sched_tables = soup.find('table', class_="schedule has-team-logos align-left")
schedules_sites = pd.read_html(sched_tables.prettify())
schedules_sites

# Load - Transform

### Load dump to mongo / Clean on the go

In [ ]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.nba
# db.nba_players_stg.drop()
# db.nba_news_live.drop()

In [ ]:
base_reference_url = "https://www.landofbasketball.com"
#Traverse to all players
for letter in string.ascii_lowercase[:26]:
    players_url = f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm"
    browser.visit(players_url)
    browser.html
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    all_player_table = soup.find_all('div', class_="indice-items")
    for row in all_player_table:
        player_d = {}
        print(row.find('a')['href'], end='')
        player_url = row.find('a')['href'].replace('..', base_reference_url)
        tables = pd.read_html(player_url)
        try_df = tables[1].set_index(0)
        try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
        try_df = try_df.to_dict('index')
        player_d['Name'] = tables[3][0][0].split(' Profile',1)[0]
        player_d['Data'] = try_df
        db.nba_players_stg.insert_one(player_d)

### Validate data availability

In [ ]:

# result_dict['facts'] = list(db.mars_fact.find({}, {"_id": 0}))
for a in db.nba_players_stg.find({}, {"_id": 0}).limit(2):
    print(a)

## Scrape News

In [ ]:
nba_news_url = "https://apnews.com/NBA"
browser.visit(nba_news_url)
browser.html
# time.sleep(10)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_tables = soup.find_all('div', attrs={"data-key":"feed-card-wire-story-with-image"})
# schedules_sites = pd.read_html(sched_tables.prettify())
# schedules_sites


In [ ]:
for row in news_tables:
    news_content = {}
    news_content['headline'] = row.h1.text
    news_content['content'] = row.p.text
    print(f"Loading {row.h1.text}....")
    db.nba_news_live.insert_one(news_content)

### ETL For elections

In [ ]:
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_pd = pd.read_json(geojson_source)

# json_file = "etl/gz_2010_us_050_00_500k.json"
# customer_location_df = pd.read_json(json_file, encoding="utf-8")

In [ ]:
results_file = "etl/countypres_2000-2016.csv"
results_df = pd.read_csv(results_file, encoding="utf-8")
results_df.head()
results_df.dropna(inplace=True)
results_df.drop(columns=['office', 'version'],inplace=True)
results_df.head()

In [ ]:
# results_df.loc[results_df['county'].str.lower() =="Orange".lower(),:]

# Cast float to integer
results_df.FIPS = results_df.FIPS.astype(int)
results_df.candidatevotes = results_df.candidatevotes.astype(int)
results_df.party = results_df.party.str.title()
# Calculate percent won
results_df["percentwon"] = ((results_df.candidatevotes / results_df.totalvotes) * 100).map("{0:,.2f}".format).astype(float)
# results_df.percentwon = results_df.percentwon.astype(int)
# Drop index
results_df.reset_index(drop=True, inplace=True)

# Test results
# results_df.loc[(results_df['county'].str.lower() =="Orange".lower()) & (results_df['state'] == "California"),:].head(100)
# results_df.head()
# results_2016 = results_df.loc[(results_df.year == 2016) & (results_df['state'] == "California"),:]
#results_2016 = results_df.loc[(results_df.year >= 2008) & (results_df.year <= 2016) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]

In [ ]:
results_2016

In [ ]:
# results_df.to_json("etl/processed.json", orient='records')

## GEO JSON BIG SOURCE

In [ ]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson" https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?refine.stusab=CA
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_source = "etl/us-county-boundaries.csv"
geojson_pd = pd.read_csv(geojson_source, delimiter=';')
geojson_pd.dtypes
geojson_pd = geojson_pd[['GEOID','Geo Shape']]

### GEOJSON Small source

In [ ]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson" https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?refine.stusab=CA
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_source = "etl/gz_2010_us_050_00_20m.json"
geojson_pd = pd.read_json(geojson_source, encoding='latin-1', lines=True)
geojson_pd.rename(columns={"geometry": "Geo Shape"},inplace=True)
# geojson_pd = geojson_pd[['Geometry']]
# geojson_pd['FIPS'] = int(geojson_pd['properties']['GEO_ID'][-5:])
geojson_pd['GEOID']  = geojson_pd.apply(lambda row: int(row.properties['GEO_ID'][-5:]), axis=1)
# geojson_pd['properties'][0]['GEO_ID'] '0500000US01001'
# int(geojson_pd['properties'][0]['GEO_ID'][-5:])
# geojson_pd

In [5]:

geojson_source = "etl/us_states.json"
geojson_pd = pd.read_json(geojson_source, lines=True)
geojson_pd


,type,geometry,properties,id
0,Feature,"{'type': 'Polygon', 'coordinates': [[[-80.519,...","{'STATE_ID': '54', 'STATE_NAME': 'West Virginia'}",54
1,Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...","{'STATE_ID': '12', 'STATE_NAME': 'Florida'}",12
2,Feature,"{'type': 'Polygon', 'coordinates': [[[-90.6428...","{'STATE_ID': '17', 'STATE_NAME': 'Illinois'}",17
3,Feature,"{'type': 'Polygon', 'coordinates': [[[-96.7802...","{'STATE_ID': '27', 'STATE_NAME': 'Minnesota'}",27
4,Feature,"{'type': 'Polygon', 'coordinates': [[[-77.7195...","{'STATE_ID': '24', 'STATE_NAME': 'Maryland'}",24
5,Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...","{'STATE_ID': '44', 'STATE_NAME': 'Rhode Island'}",44
6,Feature,"{'type': 'Polygon', 'coordinates': [[[-116.916...","{'STATE_ID': '16', 'STATE_NAME': 'Idaho'}",16
7,Feature,"{'type': 'Polygon', 'coordinates': [[[-71.5522...","{'STATE_ID': '33', 'STATE_NAME': 'New Hampshire'}",33
8,Feature,"{'type': 'Polygon', 'coordinates': [[[-81.8485...","{'STATE_ID': '37', 'STATE_NAME': 'North Caroli...",37
9,Feature,"{'type': 'Polygon', 'coordinates': [[[-73.3431...","{'STATE_ID': '50', 'STATE_NAME': 'Vermont'}",50


In [ ]:
geojson_pd.head(10)

In [ ]:
# geojson_pd
# results_df
def geo_merger(results_2016):
    joined_df = pd.DataFrame();
    joined_df = geojson_pd.merge(results_2016[['FIPS','year','state_po','county','candidate','party','candidatevotes','totalvotes','percentwon']].loc[(results_2016.party == 'Republican')], how="inner", right_on='FIPS', left_on='GEOID');
    joined_df = joined_df.merge(results_2016[['FIPS','candidate','party','candidatevotes','totalvotes','percentwon']].loc[(results_2016.party == 'Democrat')], how="inner", right_on='FIPS', left_on='GEOID',suffixes=('_rep', '_dem'));
    joined_df['winner'] = np.where(joined_df['candidatevotes_rep'] > joined_df['candidatevotes_dem'], 'red',  'blue');
    return joined_df;


In [ ]:
results_2008 = results_df.loc[(results_df.year == 2008) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]
results_2012 = results_df.loc[(results_df.year == 2012) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]
results_2016 = results_df.loc[(results_df.year == 2016) & ((results_df.party == 'Republican') | (results_df.party == 'Democrat')),:]
merged_2008 = geo_merger(results_2008);
merged_2012 = geo_merger(results_2012);
merged_2016 = geo_merger(results_2016);


In [ ]:
final_merged = pd.concat([merged_2008, merged_2012, merged_2016])

In [ ]:
final_merged

In [ ]:
def df_to_geojson(df, properties):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{}
#                    'geometry':{}
                  }
        feature['geometry'] = row['Geo Shape'] #Use json.loads(row['Geo Shape']) if using the big source
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [ ]:
# properties = ['year', 'state_po', 'county', 'candidate_rep', 'party_rep', 'candidatevotes_rep', 'percentwon_rep', 'candidate_dem', 'party_dem', 'candidatevotes_dem', 'totalvotes_dem', 'percentwon_dem', 'winner']
properties = ['year' ,'state_po', 'county', 'candidate_rep', 'party_rep', 'candidatevotes_rep', 'percentwon_rep', 'candidate_dem', 'party_dem', 'candidatevotes_dem', 'totalvotes_dem', 'percentwon_dem', 'winner']
geojson = df_to_geojson(final_merged, properties)

In [ ]:
json = json.dumps(geojson)
f = open("etl/processed.json","w")
f.write(json)
f.close()

# output_filename = 'dataset.js'
# with open(output_filename, 'wb') as output_file:
#     output_file.write('var dataset = ')
#     json.dump(geojson, output_file, indent=2) 

In [ ]:
geojson_pd